In [12]:
import os
import pandas as pd
import pickle


#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
grid_search_results = []
results_path = os.path.join('logs_mac', 'general_gradient_clipping','gridsearch_results.pkl')
with open(results_path, 'rb') as f:
    partial_results = pickle.load(f)
grid_search_results.extend(partial_results)

no_clipping_path = os.path.join('logs', 'general_gradient_clipping','gridsearch_results.pkl')
with open(no_clipping_path, 'rb') as f:
    partial_results = pickle.load(f)
grid_search_results.extend(partial_results)

print(grid_search_results[0].keys())
    
    

dict_keys(['training-optimizer_params-clipnorm', 'metrics_per_fold'])


In [13]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

9
training-optimizer_params-clipnorm: 0.1
	Fold 1:
		mean_squared_error: 0.000023 (1.539984)
			per_sample s (u):
				 [1.7092178e-05, 1.0459447e-05, 1.7114975e-05, 1.0447119e-05, 3.6143534e-05, 1.8727304e-05, 1.0586772e-05, 1.9460085e-05, 1.3254177e-05, 1.3675878e-05, 3.072154e-05, 3.64144e-05, 1.7456967e-05, 2.1737427e-05, 1.9658459e-05, 1.9762183e-05, 1.4990598e-05, 2.0105066e-05, 1.0310759e-05, 1.5017606e-05, 1.836026e-05, 1.3183872e-05, 1.0427703e-05, 1.3956721e-05, 2.2104647e-05, 7.178329e-05, 3.8056034e-05, 2.3279641e-05, 3.989845e-05, 3.6395493e-05, 3.101343e-05, 1.0508578e-05, 1.8876799e-05, 1.061431e-05, 1.4411108e-05, 4.5520264e-05, 1.4038693e-05, 1.0542182e-05, 4.3122578e-05, 1.1867052e-05, 6.8875765e-05, 1.09437005e-05, 3.5157325e-05, 2.1453388e-05, 1.0381633e-05]
				([1.1294419, 0.8386972, 1.1315671, 0.83123994, 2.064514, 1.24922, 0.86509836, 1.3074058, 1.3942165, 0.991685, 1.8048414, 2.0689483, 2.264066, 2.7676094, 1.3152549, 1.3219038, 1.0889071, 1.3439374, 0.77850336,

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [14]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    exp_name = get_exp_name(exp)
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

#for exp_name, samples in exp_samples.items():
#    print(f"{exp_name}: {len(samples)}")
#    print(f"mean: {sum(samples)/len(samples)}")
#    print(f"std: {pd.Series(samples).std()}")
    


In [5]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(9, 10)


FriedmanchisquareResult(statistic=3.8933333333333735, pvalue=0.8666167196194458)

In [6]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[0.06445312 0.16015625 0.19335938 0.23242188 0.27539062 0.27539062
 0.32226562 0.32226562 0.32226562 0.375      0.375      0.43164062
 0.43164062 0.43164062 0.4921875  0.4921875  0.4921875  0.4921875
 0.55664062 0.625      0.625      0.6953125  0.6953125  0.6953125
 0.76953125 0.76953125 0.84570312 0.84570312 0.921875   0.921875
 0.921875   0.921875   0.921875   1.         1.         1.        ]
0.064453125 is not st. significant
0.001388888888888889 is the threshold


(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.]),
 0.0014237991678133222,
 0.001388888888888889)

In [7]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Clipnorm=0.1', 'Clipnorm=0.5', 'Clipnorm=1.0', 'Clipnorm=2.0',
       'Clipnorm=5.0', 'Clipnorm=10.0', 'Clipnorm=50.0', 'Clipnorm=100.0',
       'Clipnorm=none'],
      dtype='object')

In [8]:
exp_samples.values()

dict_values([[1.2409608, 0.75423336, 0.83713555, 0.757369, 1.1228013, 1.0316198, 1.0056671, 1.0426785, 1.3374598, 0.9788956], [0.98046416, 1.2809418, 2.077669, 0.8090768, 1.7967783, 0.8824767, 0.7750812, 0.92856246, 0.8389853, 1.1018453], [1.0544313, 1.4071198, 0.81949013, 0.7480507, 1.4639963, 0.9072932, 0.91162753, 0.9135043, 1.0528244, 1.278409], [0.74009365, 1.202528, 1.3590114, 0.86525965, 0.98026633, 1.178798, 0.9236801, 1.5810298, 1.2618318, 1.1504799], [0.8406659, 0.767439, 0.91291136, 1.0718352, 0.803354, 1.7672521, 0.9321371, 0.8898348, 0.90137315, 0.77659804], [1.0589639, 0.87662435, 0.72897, 1.1410353, 1.0259005, 1.0485839, 0.85776424, 0.90264356, 1.0595093, 1.013633], [0.81598055, 0.81947345, 1.1154935, 0.97719616, 1.2673174, 0.9136546, 0.9117614, 1.1588271, 1.0880591, 0.9119259], [0.8670568, 1.4532036, 1.0197139, 0.99836516, 0.9295644, 1.1474026, 0.7079735, 1.6072615, 0.77297544, 0.885034], [0.84800893, 0.74030286, 0.8931106, 1.2917694, 0.87922287, 0.9265734, 0.7937312, 1

In [9]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

['Clipnorm=0.1', 'Clipnorm=0.5', 'Clipnorm=1.0', 'Clipnorm=2.0', 'Clipnorm=5.0', 'Clipnorm=10.0', 'Clipnorm=50.0', 'Clipnorm=100.0', 'Clipnorm=none']


In [15]:
def get_best_optimizer_by_type(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    opt_types = set([name.split(" ")[0] for name in treatment_names])
    print(opt_types)
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    best_optimizers = {}
    for name, avg_rank in sorted_treatments:
        opt_type = name.split(" ")[0]
        if opt_type not in best_optimizers:
            best_optimizers[opt_type] = (name, avg_rank)
    return best_optimizers

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

best_optimizers = get_best_optimizer_by_type(diagram)
diagram = get_partial_diagram([name for name, _ in best_optimizers.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in best_optimizers.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

diagram.to_file(
    "critdd_clipnorm_p0_01.tex",
    alpha=0.01,
    adjustment="holm",
    reverse_x=True,
)

{'Clipnorm=5.0', 'Clipnorm=0.5', 'Clipnorm=10.0', 'Clipnorm=0.1', 'Clipnorm=100.0', 'Clipnorm=50.0', 'Clipnorm=none', 'Clipnorm=1.0', 'Clipnorm=2.0'}
Previous average ranks:
Clipnorm=5.0: 4.1
Clipnorm=none: 4.3
Clipnorm=100.0: 4.9
Clipnorm=0.5: 5.0
Clipnorm=1.0: 5.0
Clipnorm=10.0: 5.0
Clipnorm=50.0: 5.1
Clipnorm=0.1: 5.4
Clipnorm=2.0: 6.2

New average ranks:
Clipnorm=5.0: 4.4622222222222225
Clipnorm=none: 4.528888888888889
Clipnorm=10.0: 4.595555555555555
Clipnorm=50.0: 4.7444444444444445
Clipnorm=100.0: 5.022222222222222
Clipnorm=0.5: 5.184444444444445
Clipnorm=0.1: 5.273333333333333
Clipnorm=1.0: 5.433333333333334
Clipnorm=2.0: 5.7555555555555555
